In [ ]:
# This code is for processing output files from the University of Wyoming 
# radiosonde record. It assumes that all of the records are in a single
# folder. Each file has up to a year's worth of observations and each
# file must strictly adhere to the UW archive format, as this code assumes
# things like the number of spaces between columns and the number of metadata
# rows at the end.
#
# getIDLines(path to single file) --> list of the indices of the first lines of 
#                                     individual soundings
#
#
# fetchSingleRecord(path to a single file, beginning row, ending row) 
#                                 --> dataframe with one data from one sounding, 
#                                     and a dictionary with metadata
#
# computeInversionQuantities(dataframe with sounding, metadata) 
#                                 -->  dictionary with base_height, strength, 
#                                      depth, and length (length being the number
#                                      of observations)
#
# main(name of file to create, path to data file) --> CSV with inversion values
#
# Additional filters could go into main()

import os
import numpy as np
import pandas as pd

# 'Radiosonde Data/' + f_list entry = fname
def getIDLines(fname):
    """Each text file has up to a year's worth of radiosonde
    profiles. The record analyzer only can handle a single
    sounding at a time. So we need to find the start and stop lines
    for each sounding. fname is the path to the file to analyze."""
    
    fid = open(fname)
    lines = fid.readlines() # returns a list containing all lines of text from fname
    station_id = lines[0].split()[0]
    
    id_loc = []
    for x in range(0,len(lines)):
        # The reason for the try/except thing is that if consists only of white space
        # or has a single entry then the return from the .split() will not be subsettable.
        # In that case, it doesn't matter, since it also won't be a header line
        try:
            if lines[x].split()[0] == station_id:
                id_loc.append(x)
        except IndexError:
            pass
        
    return id_loc + [len(lines)]



# Function to process a single sounding.

def fetchSingleRecord(fname, header_row, end_row):
    """Reads and processes a single sounding from the U Wyoming 
    weather station data. It is important that the sounding be in
    the exact UW format. Since output data can have a full year worth
    of soundings in a single file, specifying the line number where the header
    is located (header_row) and the line number of the next sounding's header
    (end_row) is necessary. Output is a dataframe with the information from the
    sounding table, but not including the analysis or metadata. 
    
    Much of this code is directly copied from the SkewT python package.
    """

    with open(fname) as fid:
        lines = fid.readlines() # returns a list containing all lines of text from fname
    
    # Spacing is consistent, these are the left and right sides of the columns
    lhi=[1, 9,16,23,30,37,46,53,58,65,72]
    rhi=[7,14,21,28,35,42,49,56,63,70,77]

    # initialise output data structure
    output={}

    header=lines[header_row].split()
    station_id = header[0]
    date = pd.to_datetime(' '.join(header[-3:] +  [header[-4]]))
    fields=lines[header_row + 2].split()
    units=lines[header_row + 3].split()

    for ff in fields:
        output[ff]=[] # Initialize container for fields
    
        
        # For each of the lines with observations, split the lines and save
        # the data as float in the output dictionary. The last 25 lines are always
        # the calculations and metadata.
        
    for line,idx in zip(lines[header_row + 5:],range(header_row, end_row-25)):
  
        # If pressure field is missing, skip
        try: 
            output[fields[0]].append(float(line[lhi[0]:rhi[0]]))
        except ValueError: 
            break # I think this would just go to the next line 
            # This may be the time to ID the line "Station information.."

        for ii in range(1,len(rhi)):
            try: 
                textdata=line[lhi[ii]:rhi[ii]].strip()
                output[fields[ii]].append(float(textdata))
            except ValueError: 
                output[fields[ii]].append(np.NaN)
        

    # Create Pandas dataframe 
    df = pd.DataFrame(columns=fields, index=np.arange(0,len(output[fields[0]])))

    for ff in fields:
        df[ff] = output[ff]
    
    # Now grab the metadata:
    station_metadata = {}
    station_metadata['date'] = date
    still_looking = True
    i = header_row
    while still_looking: 
        # Find the second header so we can count backwards
        if lines[i].split()[0] == 'Station':
            if lines[i].split()[1] == 'number:':
                # could have backup loop to ensure no errors here, too
                station_metadata['number'] = lines[i].split()[2]
                station_metadata['latitude'] = float(lines[i+2].split()[2])
                station_metadata['longitude'] = float(lines[i+3].split()[2])
                station_metadata['elevation'] = float(lines[i+4].split()[2])
                still_looking = False
        i += 1
    
    return (df, station_metadata)

def computeInversionQuantities(df,meta):
    """Calculate, if present, the inversion base height,
    inversion depth, and inversion strength for a sounding 
    stored in df. Df and meta are the output from the fetchSingleRecord
    function. Records number of measurements in the sounding.
    Returns a dictionary with keys base_height, depth, strength, and length."""
    
    # Todo: require inversions to be less than a certain height
    # Inversion base must be below 700 hPa
    # Todo: allow for small non-inversion layers
    # Threshold is 100 m
    # Vector with temperatures must have at least 10 observations.
    
    # Only include observations above or equal to the station elevation.
    valid = df.HGHT >= meta['elevation']
    tvec1 = np.array(df.TEMP[valid])
    zvec1 = np.array(df.HGHT[valid])
    pvec1 = np.array(df.PRES[valid])
    
    # Drop missing obvservations
    valid = ~(np.isnan(tvec1) | np.isnan(zvec1))
    tvec = tvec1[valid]
    zvec = zvec1[valid]
    pvec = pvec1[valid]
    
    # If there are insufficient observations to compute gradient, return NaNs.
    length = len(tvec)
    if length <= 2:
        return {'base_height':np.NaN, 'depth':np.NaN, 'strength':np.NaN,  
                'length':len(tvec), 'base_press':np.NaN}
   

    # Next we look for an inversion
    diffsign = np.sign(np.diff(tvec))
    i = 0
    inversion_present = True
    
    while diffsign[i] < 0:
    # This will find, if present, the base height of the inversion layer
    # If no inversion is present, then it will be flagged
        i += 1
        if i == len(diffsign):
            inversion_present = False
            break

    # If there is indeed an inversion, find the base height, depth, and strength
    if inversion_present:
        base_height = zvec[i] - zvec[0] # There are a couple cases where this is negative... how?
        if base_height < 0:
            print(meta['number'])
        base_press = pvec[i]
        
        # Now look for the top of the inversion
        # The inversion top is where diffsign < 0.
        # If diffsign < 0, check if the next layer is an inversion layer
        for j in range(i,len(diffsign)):
            if diffsign[j] < 0:
                if (j+2 < len(diffsign)):
                    if (diffsign[j+1] > 0) & (zvec[j+2]-zvec[j+1] < 100):
                        continue
                    else:
                        depth = zvec[j] - zvec[i]
                        strength = tvec[j] - tvec[i]
                        break
                else:
                    depth = zvec[j] - zvec[i]
                    strength = tvec[j] - tvec[i]
                    break
                        
        if j == len(diffsign)-1: 
            if diffsign[j] >= 0: # Then inversion top is beyond the last observation
                depth = np.NaN
                strength = np.NaN    
                
    # If no inversion is present, then the variables are undefined
    else:
        base_height, depth, strength, base_press = np.NaN, np.NaN, np.NaN, np.NaN
    
    return {'base_height':base_height, 'depth':depth, 'strength':strength, 'length':length,'base_press':base_press}
   

def main(datapath, savepath):
    # Datapath: path to folder containing radiosonde soundings
    # savepath: path + filename for new file

    f_list = os.listdir(datapath)
    f_list = f_list[1:] # The first item in the list is always '.DS_Store', which we don't need.

    # Initialize the pandas dataframe
    columns = ['date', 'elevation', 'latitude', 
               'longitude','number','base_height', 
               'depth','strength','length']

    big_df = pd.DataFrame(np.NaN, columns=columns,index=[0])
    big_df = big_df[columns] # ensure that the column order is correct
    big_df.to_csv(savepath, header=True, index=False)
    
    idx = 1

    for fname in f_list:
        id_loc = getIDLines(datapath + '/' + fname)
        for i in range(0,len(id_loc)-1):
            df, meta = fetchSingleRecord(datapath + '/' + fname, header_row=id_loc[i], end_row=id_loc[i+1])
            inv_df = computeInversionQuantities(df,meta)
            small_df = pd.DataFrame(dict(inv_df, **meta), index=[idx])
            small_df = small_df[columns] # ensure columns in right order
            big_df = pd.concat([big_df, small_df])
            idx += 1

        # Write result after each year
        with open('radsonde_inv_analysis.csv','a') as f:
            big_df.drop([0]).to_csv(f,header=False,index=False)

        # Reset big_df
        big_df = pd.DataFrame(np.NaN, columns=columns,index=[0])

datapath = '/Users/watkdani/Google Drive/Research/Radiosonde Analysis/Test Data'
savepath = '/Users/watkdani/Google Drive/Research/Radiosonde Analysis/test_data.csv'
main(datapath, savepath)

In [ ]:
# Creating lookup table for the radiosonde data, this can later be joined with the radsonde analysis csv
def read_station_list(fname):
    """Read the weather station code table and put into a dataframe."""
    import numpy.ma as ma
    import numpy as np
    import pandas as pd

    with open(fname) as fid:
        lines = fid.readlines() # returns a list containing all lines of text from fname
    
    # Spacing is consistent, these are the left and right sides of the columns
    columns = ['call','number','name','st/pr','country','latitude','longitude','elevation']
    lhi=[0, 10, 16, 49, 52, 55, 62, 69]
    rhi=[4, 15, 48, 51, 54, 60, 67, 73]

    # initialise output data structure
    df = pd.DataFrame(columns=columns, index=np.arange(0,len(lines)))
     
    for line,idx in zip(lines,range(0, len(lines))):
  
        
        try: 
            output[columns[0]].append(float(line[lhi[0]:rhi[0]]))
        except ValueError: 
            break # I think this would just go to the next line 
            # This may be the time to ID the line "Station information.."

        for ii in range(1,len(rhi)):
            try: 
                textdata=line[lhi[ii]:rhi[ii]].strip()
                output[fields[ii]].append(float(textdata))
            except ValueError: 
                output[fields[ii]].append(np.NaN)
        

    for ff in fields:
        df[ff] = output[ff]
    
    # Now grab the metadata:
    station_metadata = {}
    station_metadata['date'] = date
    still_looking = True
    i = header_row
    while still_looking: 
        # Find the second header so we can count backwards
        if lines[i].split()[0] == 'Station':
            if lines[i].split()[1] == 'number:':
                # could have backup loop to ensure no errors here, too
                station_metadata['number'] = lines[i].split()[2]
                station_metadata['latitude'] = float(lines[i+2].split()[2])
                station_metadata['longitude'] = float(lines[i+3].split()[2])
                station_metadata['elevation'] = float(lines[i+4].split()[2])
                still_looking = False
        i += 1
    
    return (df, station_metadata)

In [329]:
import numpy.ma as ma
import numpy as np
import pandas as pd
fname = '/Users/watkdani/Google Drive/Research/Radiosonde Analysis/snstns.tbl'
with open(fname) as fid:
    lines = fid.readlines() # returns a list containing all lines of text from fname

# Spacing is consistent, these are the left and right sides of the columns
columns = ['call','number','name','st/pr','country','latitude','longitude','elevation']
lhi=[0, 10, 16, 49, 52, 55, 62, 69]
rhi=[4, 15, 48, 51, 54, 60, 67, 73]

# initialise output data structure
df = pd.DataFrame(columns=columns, index=np.arange(0,len(lines)))


for line,idx in zip(lines,range(0, len(lines))):
    
    for ii in range(len(columns)):
        df.loc[idx,columns[ii]] = line[lhi[ii]:rhi[ii]]
    
#     if len(line.split()) == 9: # nothing missing
#         df.loc[idx, columns] = line.split()[0:8]
#     elif len(line.split()) == 8: # no call sign. 
#         df.loc[idx, columns[1:]] = line.split()[0:7]
#     else:
#         pass
    
df.loc[df['st/pr'] == '--', 'st/pr'] = np.NaN
for idx in range(len(lines)):
#     latstring = df.loc[idx,'latitude']
#     latfloat = float(latstring[0:-2].strip() + '.' + latstring[-2:].strip())
#     df.loc[idx,'latitude'] = latfloat
#     lonstring = df.loc[idx,'longitude']
#     lonfloat = float(lonstring[0:-2].strip() + '.' + lonstring[-2:].strip())
#     df.loc[idx,'longitude'] = lonfloat
    df.loc[idx,'latitude'] = convert_degree_string_to_float(df.loc[idx,'latitude'])
    df.loc[idx,'longitude'] = convert_degree_string_to_float(df.loc[idx,'longitude'])


In [328]:
def convert_degree_string_to_float(dmstr):
    """ Lat and lon are strings in the format (s)DDdd and (s)DDDdd where (s) is 
    either ' ' or '-', and need to be converted to DD.dd / DDD.dd respectively.
    """
    DD = dmstr[0:-2]
    dd = dmstr[-2:]
    if len(DD.strip()) > 0:
        numeric_degrees = float(DD.strip() + '.' + dd)
    elif dd.strip()[0] == '-':
        numeric_degrees = -float('.' + dd.strip()[1:])
    else:
        numeric_degrees = float('.' + dd.strip())
    return numeric_degrees

In [338]:
df.loc[(df['st/pr']=='AK') & (df['country']=='US'),]

,call,number,name,st/pr,country,latitude,longitude,elevation
1011,PABR,70026,BARROW/W._POST_W.ROGERS,AK,US,71.28,156.79,19
1012,,70027,NORTH SLOPE,AK,US,71.32,156.62,8
1013,PAOT,70133,"KOTZEBUE,_RALPH_WIEN",AK,US,66.86,162.63,5
1014,,70197,CENTRAL,AK,US,65.48,144.66,252
1015,PAOM,70200,NOME,AK,US,64.5,165.43,7
1016,PABE,70219,BETHEL/BETHEL_AIRPORT,AK,US,60.78,161.84,33
1017,PAMC,70231,MCGRATH,AK,US,62.96,155.61,103
1018,,70252,TALKEETNA,AK,US,62.3,150.41,151
1019,PAFA,70261,FAIRBANKS/INT,AK,US,64.81,147.88,134
1020,PABI,70266,FORT_GREELY,AK,US,63.96,145.7,398


In [340]:
df.to_csv('station_list.csv',header=True, index=False)